In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from algorithms.MAB import MAB
from algorithms.reward_model import LinearRewardModel, EqualGapRewardModel
from algorithms.bai.tas import TaS
from algorithms.bai.uniform import UniformBAI
from algorithms.fairness_model import ProportionalFairnessModel, VectorFairnessModel, make_prespecified_fairness_model
from tqdm import tqdm
import pickle
from scipy.stats import bootstrap
import warnings
import seaborn as sns
import matplotlib.ticker as mtick

plt.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
plt.rc('text', usetex=True)
SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def compute_confidence(x):
    mean = np.mean(x)
    ci = bootstrap((x,), statistic=np.mean).confidence_interval
    ci = (np.nan_to_num(ci.low, nan =0), np.nan_to_num(ci.high, nan =0))
    return mean, ci


def compute_confidence_violation(x):
    z = x.mean(0)
    mean_ci = bootstrap((z,), statistic=np.mean).confidence_interval
    max_ci = bootstrap((z,), statistic=np.max).confidence_interval
    min_ci = bootstrap((z,), statistic=np.min).confidence_interval

    y = x.min(-1)
    mean_violation = bootstrap((y,), statistic=np.mean).confidence_interval


    return mean_ci, max_ci, min_ci, mean_violation

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Or if you are using > Python 3.11:
with warnings.catch_warnings(action="ignore"):
    fxn()

In [ ]:
SEED = 5
np.random.seed(SEED)
K = 10
R_MAX = K // 2.5
p0_linear = 0.9
p0_equalgap = 0.99
SOLVER = cp.ECOS
NUM_CPUS = 5
DELTAS = [1e-3, 1e-2, 1e-1]
N_SIM = 100

models = [
        ('Linear gaps, agonistic fairness', LinearRewardModel(R_MAX = R_MAX, K=K), lambda _: make_prespecified_fairness_model(w, p0_linear, agonistic=True), 'prespecified_linear_gaps_agonistic'),
        ('Linear gaps, antagonistic fairness', LinearRewardModel(R_MAX = R_MAX, K=K), lambda _: make_prespecified_fairness_model(w,  p0_linear, agonistic=False), 'prespecified_linear_gaps_antagonistic'),
        ('Equal gaps, agonistic fairness', EqualGapRewardModel(R_MAX = R_MAX//2, K=K), lambda w: make_prespecified_fairness_model(w,  p0_equalgap, agonistic=True),'prespecified_equal_gaps_agonistic'),
        ('Equal gaps, antagonistic fairness', EqualGapRewardModel(R_MAX = R_MAX//2, K=K), lambda w: make_prespecified_fairness_model(w,  p0_equalgap, agonistic=False),'prespecified_equal_gaps_antagonistic')
    ]

fig, ax = plt.subplots(1,4, figsize=(16,4), sharey=True)
idx = 0
for idx, (name, reward_model, fairness_model, filename) in enumerate(models):
    instance = MAB(reward_model=reward_model, fairness_model=None)
    [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)

    instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
    [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

    ax[idx].plot(np.arange(1,K+1),w, 'k', label = r"$w^\star$")
    ax[idx].plot(np.arange(1,K+1),w_fair, "g--",label = r"$w_{p}^\star$")
    ax[idx].stem(np.arange(1,K+1), instance.f(instance.THETA), "r--", label = r"Fairness $p(\theta)$")
    ax[idx].grid(linestyle = ":")
    ax[idx].set_xlabel(r'Arms $a$')
    print(w)
    print(f"[{name}]: T*_{{theta}} = {sol}\nT*_{{theta,p}} = {sol_fair}\n p = {instance.f(instance.THETA)} - psum: {instance.f(instance.THETA).sum()} - pmin: {instance.f(instance.THETA).min()}")
    print('------------------------------------')
    
    idx += 1

ax[0].set_title('Linear gaps, agonistic fairness')
ax[1].set_title('Linear gaps, antagonistic fairness')
ax[2].set_title('Equal gaps, agonistic fairness')
ax[3].set_title('Equal gaps, antagonistic fairness')
ax[0].set_ylabel('Probability')
ax[0].legend()
plt.suptitle('Pre-specified rates')
plt.savefig('images/bai/synthetic/prespecified_allocation.pdf', bbox_inches='tight')

In [ ]:
box_width = 0.5
points_width = 0.1
alpha_val = 0.3
colors = ["r","g","b"]


for delta in DELTAS:
    fig, ax = plt.subplots(1, 4, figsize=(16,4))
    for idx, (name, reward_model, fairness_model, filename) in enumerate(models):
        print(f'------ [{delta}]: {filename} ------')
        with open(f'data/synthetic/delta_{delta}/{filename}_{delta}.pkl', 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)
        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)

        instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
            
        

        for idx_algo, algo_name in enumerate(results.keys()):
            mu_tau, ci_tau = compute_confidence(results[algo_name]['t_vec'])
            mu_perr, ci_err = compute_confidence(results[algo_name]['p_err'])
            print(f"[{algo_name}] Sample complexity: {mu_tau} ({ci_tau}) - P(err): {mu_perr} ({ci_err})")


            ax[idx].scatter(idx_algo+1+np.random.rand(len(results[algo_name]['t_vec']))*points_width, results[algo_name]['t_vec'], alpha = alpha_val , c = colors[idx_algo])
            ax[idx].boxplot(results[algo_name]['t_vec'],positions = [idx_algo+1],showfliers=False, widths=(box_width), showmeans=True)

            

        ax[idx].plot(np.arange(6),np.ones(6)*sol*np.log(1/delta),"b--",label = "LB")
        ax[idx].plot(np.arange(6),np.ones(6)*sol_fair*np.log(1/delta),"r--",label = "LB Fair")
        ax[idx].set_xticks([k+1 for k in range(len(results.keys()))], results.keys())
        ax[idx].set_xlim([0.5, len(results.keys()) + 0.5])
        #plt.yscale('log')
        #ax[idx].legend()
        ax[idx].set_title(f'{name}')
        ax[idx].grid(linestyle = ":")
    ax[0].legend()
    plt.suptitle(f'Pre-specified rates - Sample complexity - $\\delta = {delta}$')
    plt.savefig(f'images/bai/synthetic/prespecified_sample_complexity_{delta}.pdf', bbox_inches='tight')

    plt.show()

In [ ]:
colors = ["r","g","b"]


for delta in DELTAS:
    idx=0
    fig, ax = plt.subplots(1,2, sharey=False, figsize=(8,3.5))
    for _, (name, reward_model, fairness_model, filename) in enumerate(models):
        if 'Equal' in name or 'equal' in name: continue
        print(f'------ [{delta}]: {filename} ------')
        with open(f'data/synthetic/delta_{delta}/{filename}_{delta}.pkl', 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)

        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)
        instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

        mask = instance.f(instance.THETA) > 0

        for idx_algo, algo_name in enumerate(results.keys()):
            w_fair_vec = results[algo_name]['w_vec']

            fairness_res_fbai = np.array(
                [max(0,np.max(instance.f(instance.THETA) - sample))  for x in w_fair_vec  for t, sample in enumerate(x)])

            sns.kdeplot(fairness_res_fbai, label=algo_name, ax=ax[idx], fill=True, alpha=0.3, color=colors[idx_algo])#,bw_adjust=.2)
        
        ax[idx].set_xlabel(r'Fairness Violation')
        ax[idx].set_ylabel(r'Density' if idx == 0 else '')
        ax[idx].set_title('Pre-specified agonistic' if idx == 0 else 'Pre-specified antagonistic')
        ax[idx].grid(linestyle = ":")
        ax[idx].xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
        idx+= 1
    #plt.suptitle(f'Pre-specified Fairness - $\\delta={delta}$')
    plt.legend()

    plt.savefig(f"images/BAI/synthetic/prespecified_violation_{delta}.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
colors = ["r","g","b"]


for delta in DELTAS:
    idx=0
    fig, ax = plt.subplots(1,4, sharey=False, figsize=(16,4))
    for _, (name, reward_model, fairness_model, filename) in enumerate(models):
        print(f'------ [{delta}]: {filename} ------')
        with open(f'data/synthetic/delta_{delta}/{filename}_{delta}.pkl', 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)

        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)
        instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

        mask = instance.f(instance.THETA) > 0

        for idx_algo, algo_name in enumerate(results.keys()):
            w_fair_vec = results[algo_name]['w_vec']

            fairness_res_fbai = np.array(
                [max(0,np.max(instance.f(instance.THETA) - sample))  for x in w_fair_vec  for t, sample in enumerate(x)])

            sns.kdeplot(fairness_res_fbai, label=algo_name, ax=ax[idx], fill=True, alpha=0.3, color=colors[idx_algo])#,bw_adjust=.2)
        
        ax[idx].set_xlabel(r'Fairness Violation')
        ax[idx].set_ylabel(r'Density' if idx == 0 else '')
        title = '{} gaps, {} fairness'.format('Linear' if 'Linear' in name else 'Equal', 'Antagonistic' if 'antagonistic' in name else 'agonistic')
        ax[idx].set_title(title)
        ax[idx].grid(linestyle = ":")
        ax[idx].xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
        idx+= 1
    plt.suptitle(f'Pre-specified Fairness - $\\delta={delta}$')
    plt.legend()

    plt.savefig(f"images/BAI/synthetic/prespecified_violation_{delta}_full.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
for delta in DELTAS:
    for idx, (name, reward_model, fairness_model, filename) in enumerate(models):
        print(f'------ [{delta}]: {filename} ------')
        with open(f'data/synthetic/delta_{delta}/{filename}_{delta}.pkl', 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)

        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)
        instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

        mask = instance.f(instance.THETA) > 0

        for algo_name in results.keys():
            w_fair_vec = results[algo_name]['w_vec']

            def tolerant_mean(arrs):
                lens = [len(i) for i in arrs]
                arr = np.ma.empty((np.max(lens),len(arrs)))
                arr.mask = True
                for idx, l in enumerate(arrs):
                    arr[:len(l),idx] = l
                return arr.mean(axis = -1), arr.std(axis=-1, ddof=1)


            Z_vec_list = [[] for _ in range(K)]
            for n in range(N_SIM):
                for c in range(K):
                    if mask[c] == False: continue
                    Z_arr = np.array(w_fair_vec[n]- instance.f(instance.THETA))
                    Z_arr[Z_arr<0] = 0
                    Z_arr = np.abs(Z_arr)
                    Z_vec_list[c].append(list(Z_arr[:, c]))

            y_vec = []
            err_vec = []
            for c in range(K):
                if mask[c] == False: continue
                y, error = tolerant_mean(Z_vec_list[c])
                y_vec.append(y.data)
                err_vec.append(error.data)
            y = np.mean(np.array(y_vec),axis = 0)

            err = np.mean(np.array(err_vec),axis = 0)
            plt.plot(y,label =algo_name)
            plt.fill_between(np.arange(len(y)) + 1, y-err, y+err, alpha = 0.1,label='_nolegend_')
        plt.legend()
        plt.show()


In [ ]:

def compute_confidence(x):
    z = x.mean(0)
    mean_ci = bootstrap((z,), statistic=np.mean).confidence_interval
    max_ci = bootstrap((z,), statistic=np.max).confidence_interval
    min_ci = bootstrap((z,), statistic=np.min).confidence_interval

    y = x.min(-1)
    mean_violation = bootstrap((y,), statistic=np.mean).confidence_interval


    return mean_ci, max_ci, min_ci, mean_violation


for idx, (name, reward_model, fairness_model, filename) in enumerate(models):
    with open(f'data/{filename}.pkl', 'rb') as f:
        results = pickle.load(f)
    instance = MAB(reward_model=reward_model, fairness_model=None)

    [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)
    instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
    [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

    mask = instance.f(instance.THETA) > 0
    print(f'--------- {name} ---------')

    for _, algo_name in enumerate(results.keys()):
        w_fair_vec = results[algo_name]['w_vec']
        fairness_res_fbai = np.array([x[-1] for x in w_fair_vec]) - instance.f(instance.THETA)[None,:]
        fair_bai_res = compute_confidence(fairness_res_fbai)
        print('-------------------------------------')
        print(f'{algo_name}')
        print(f'Avg(E[N_a(tau)]/E[tau] - p(theta)) in ({fair_bai_res[0].low}, {fair_bai_res[0].high}) with 95% confidence')
        print(f'max_a E[N_a(tau)]/E[tau] - p(theta) in ({fair_bai_res[1].low}, {fair_bai_res[1].high}) with 95% confidence')
        print(f'min_a E[N_a(tau)]/E[tau] - p(theta) in ({fair_bai_res[2].low}, {fair_bai_res[2].high}) with 95% confidence')
        print(f'E[min_a N_a(tau)/tau] - p(theta) in ({fair_bai_res[3].low}, {fair_bai_res[3].high}) with 95% confidence')